# microKanren Implementation

- http://webyrd.net/scheme-2013/papers/HemannMuKanren2013.pdf
- https://github.com/ashton314/muKanren_reading/blob/master/kanren.rkt (annotated Racket version)
- I modified ashton314's version slightly to use a simpler `unify` taken from the original muKanren paper

"The entire system is comprised of a handful of functions for the implementation of variables, streams themselves, the interface for creating and combining streams, and four primitive goal constructors." --muKanren paper

In [60]:
(require racket/set) ; need to deduplicate at the end

In [59]:
(struct var (name) #:transparent)
(define var=? equal?)

;;                    substitution assoc list: variable → var-or-value
;;                    |    fresh variable counter
;;                    ↓    ↓
(define empty-state '(() . 0))

;; find a variable's value in the substitution
(define (walk varname subst)
  (let ([var-or-val (and (var? varname)
                         (assp (lambda (v) (var=? varname v)) subst))])
    (if var-or-val                      ; Was it a variable we got, and did it have a
        (walk (cdr var-or-val) subst)   ; reference? If yes, follow the new variable.
        varname)))                      ; Otherwise, we have a value OR an unbound var.

;; Needed because Racket doesn't implement R6RS's assp function
;; natively afaik
(define (assp ? lst)
  (if (null? lst)
      #f
      (if (? (caar lst))
          (car lst)
          (assp ? (cdr lst)))))

(define (occurs-check x v subst)
  (let ([v (walk v subst)])
    (if (var? v)
        (var=? v x)
        (and (pair? v) (or (occurs-check x (car v) subst)
                           (occurs-check x (cdr v) subst))))))

(define (extend-subst var val subst)
  (if (occurs-check var val subst)
      #f
      (cons (cons var val) subst)))

;;; Goal constructors

(define (== u v)
  (lambda (subst/counter)
    (let ([newsubst (unify u v (car subst/counter))])
      (if newsubst
          ;; If we did get a new subst list, keep the old counter, but
          ;; put this new subst set onto the state.
          (unit (cons newsubst (cdr subst/counter)))
          mzero))))

;; Ah, yes, a monad
(define (unit subst/counter) (cons subst/counter mzero))
(define mzero '())

;; took this version of `unify` from the original paper
(define (unify u v s)
  (let ((u (walk u s)) (v (walk v s)))
    (cond
      [(and (var? u) (var? v) (var=? u v)) s]
      [(var? u) (extend-subst u v s)]
      [(var? v) (extend-subst v u s)]
      [(and (pair? u) (pair? v))
       (let [(s (unify (car u) (car v) s))]
         (and s (unify (cdr u) (cdr v) s)))]
      [else (and (eqv? u v) s)])))

;; goal :: state -> state*
;; call/fresh :: (var -> goal) -> goal
(define (call/fresh fn)
  (lambda (subst/counter)
    (let ([counter (cdr subst/counter)])
      ((fn (var counter))
       (cons (car subst/counter) (+ counter 1))))))

(define (disj goal1 goal2)
  (lambda (subst/counter)
    ;; add the results of two streams of states
    (mplus (goal1 subst/counter)
           (goal2 subst/counter))))

(define (conj goal1 goal2)
  (lambda (subst/counter)
    ;; thread stream of states from running one goal through another
    (bind (goal1 subst/counter)
          goal2)))

;; mplus: like zip-list but for streams (which may be lazy)
(define (mplus stream1 stream2)
  (cond
    [(null? stream1) stream2]
    ;; handle the case where stream1 is a lazy stream; note how we
    ;; flip the order of the streams so we interleave them
    [(procedure? stream1) (lambda () (mplus stream2 (stream1)))]
    [else (cons (car stream1) (mplus stream2 (cdr stream1)))]))

;; bind: like map for a stream of states with a goal
(define (bind $stream goal)
  (cond
    [(null? $stream) mzero]
    ;; force the thunk and run again
    [(procedure? $stream) (lambda () (bind ($stream) goal))]
    ;; join the stream of states from running the goal on the first
    ;; state in the input stream with the result of running on the
    ;; rest of the states in the input stream
    [else (mplus (goal (car $stream))
                 (bind (cdr $stream) goal))]))

;;; Extentions

(define-syntax Zzz
  (syntax-rules ()
    ;; This is the inverse-η-delay abstracted in a macro
    [(_ goal) (lambda (subst/counter) (lambda () (goal subst/counter)))]))

(define-syntax conj+
  (syntax-rules ()
    [(_ g) (Zzz g)]
    [(_ g0 gs ...) (conj (Zzz g0) (conj+ gs ...))]))

(define-syntax disj+
  (syntax-rules ()
    [(_ g) (Zzz g)]
    [(_ g0 gs ...) (disj (Zzz g0) (disj+ gs ...))]))

(define-syntax conde
  ;; call like
  ;;
  ;; (define (father p s)
  ;;   (conde ((== p 'paul) (== s 'jason))
  ;;          ((== p 'john) (== s 'henry))
  ;;          ((== p 'jason) (== s 'tom))
  ;;          ((== p 'peter) (== s 'brian))
  ;;          ((== p 'tom) (== s 'peter))))
  ;;
  ;; it is `or' applied across the subgroups of goals which are joined by `and'
  (syntax-rules ()
    [(_ (g gs ...) ...) (disj+ (conj+ g gs ...) ...)]))

(define-syntax fresh
  (syntax-rules ()
    [(_ () g gs ...) (conj+ g gs ...)]
    [(_ (x xs ...) g gs ...)
     (call/fresh (lambda (x) (fresh (xs ...) g gs ...)))]))

;;; Utilities to force streams
(define (pull $stream)
  (if (procedure? $stream)
    (pull ($stream))
    $stream))

(define (take n $stream)
  (if (zero? n) '()
      (let ([$stream (pull $stream)])
        (cond
          [(null? $stream) '()]
          [else (cons (car $stream) (take (- n 1) (cdr $stream)))]))))

(define (take-all $stream)
  (let ([$stream (pull $stream)])
    (if (null? $stream) '() (cons (car $stream) (take-all (cdr $stream))))))

;;; Reification utilities
(define (mK-reify s/c*)
  (map reify-state/1st-var s/c*))

(define (reify-state/1st-var s/c)
  (let ([v (walk* (var 0) (car s/c))])
    (walk* v (reify-s v '()))))

(define (reify-s v s)
  (let ([v (walk v s)])
    (cond
      [(var? v)
       (let ([n (reify-name (var-name v))])
         (cons (cons v n) s))]
      [(pair? v) (reify-s (cdr v) (reify-s (car v) s))]
      [else s])))

(define (reify-name n)
  (if (symbol? n)
      n
      (string->symbol (string-append "_" "." (number->string n)))))

(define (walk* v s)
  (let ([v (walk v s)])
    (cond
      [(var? v) v]
      [(pair? v) (cons (walk* (car v) s)
                       (walk* (cdr v) s))]
      [else v])))

(define (call/empty-state g) (g empty-state))

;;; Use these to wrap your queries: run takes a number of n solutions
;;; to the program, while run* calls `take-all'.
(define-syntax run
  (syntax-rules ()
    [(_ n (xs ...) g gs ...)
     (mK-reify (take n (call/empty-state
                        (fresh (xs ...) g gs ...))))]))

(define-syntax run*
  (syntax-rules ()
    [(_ (xs ...) g gs ...)
     (mK-reify (take-all (call/empty-state
                          (fresh (xs ...) g gs ...))))]))

In [57]:
(define (parent c p)
  (conde
   [(== c 'teddy) (== p 'saraih)]
   [(== c 'andrew) (== p 'steve)]
   [(== c 'brook) (== p 'steve)]
   [(== c 'caroline) (== p 'steve)]
   [(== c 'peter) (== p 'steve)]
   [(== c 'steve) (== p 'bill)]
   [(== c 'roger) (== p 'bill)]
   [(== c 'will) (== p 'roger)]
   [(== c 'andy) (== p 'roger)]
   [(== c 'earnest) (== p 'roger)]
   [(== c 'jack) (== p 'bill)]
   [(== c 'anne) (== p 'john)]
   [(== c 'danni) (== p 'john)]
   [(fresh (s) (married p s) (parent c s))]
   [(fresh (s) (married s p) (parent c s))]))

(define (married h w)
  (conde
   [(== h 'steve) (== w 'anne)]
   [(== h 'nate) (== w 'danni)]
   [(== h 'bill) (== w 'katie)]
   [(== h 'john) (== w 'kitty)]
   [(== h 'andrew) (== w 'saraih)]))

(define (grandparent g s)
  (fresh (p) (parent g p) (parent p s)))

(define (cousin c1 c2)
  (fresh (gp)
         (grandparent c1 gp)
         (grandparent c2 gp)))

;; (run 20 (rel p c) (conj+ (grandparent c p) (== (cons c p) rel)))
;; (list->set (run 10 (gp) (grandparent 'andrew gp)))

;; unfortunately we need to gather 93 results until we get all the
;; answers for bill's grandchildren:
;; (list->set (run 93 (c) (grandparent c 'bill)))

(define (fav-num n)
  (disj (== n 42)
        (== (cons '? even?) n)))

In [58]:
(list->set (run 93 (c) (grandparent c 'bill)))

(set 'caroline 'peter 'earnest 'andy 'will 'andrew 'brook)

In [61]:
(define appendo
  (lambda (l s out)
    (disj
     (conj (== '() l) (== s out))
     (call/fresh
      (lambda (a)
        (call/fresh
         (lambda (d)
           (conj
            (== `(,a . ,d) l)
            (call/fresh
             (lambda (res)
               (conj
                (== `(,a . ,res) out)
                (lambda (s/c)
                  (lambda ()
                    ((appendo d s res) s/c))))))))))))))

In [104]:
(define (query_appendo list1 list2 list3 result)
    (appendo list1 list2 result))

In [105]:
(run* (q) (query_appendo '(1 2) '(3 4) '(1 2 3 4) q))

'((1 2 3 4))